In [ ]:
#Importing and Modifying Training & Testing Data
#SEE citations in other two files

#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

#Modules
import gensim.models
import pandas as pd
import torch
import numpy as np
from gensim.models import FastText, Word2Vec, KeyedVectors

#Deepti Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_test.csv')

drive.mount("/content/drive", force_remount=True)
modelemb = Word2Vec.load('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/w2v_OA_CR_300d.bin')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

#In training dataset, make each class have 1100 abstract samples to even out classes
dataset = dataset[dataset[:, 0].argsort()]
datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]
  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]
dataset=datasetnew

In [ ]:
pip install gensim

In [ ]:
import gensim.models

In [ ]:
#Importing and Modifying Training & Testing Data

from google.colab import drive
drive.mount('/content/drive')

#modules
import pandas as pd
import torch
import numpy as np
from gensim.models import FastText, Word2Vec, KeyedVectors

#Sanjana Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_test.csv')

drive.mount("/content/drive", force_remount=True)

modelemb = Word2Vec.load('/content/drive/MyDrive/Deep Learning Project 2/w2v_OA_CR_300d.bin')
modelemb.wv.has_index_for('diabetes')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

#in training dataset, make each class have 1100 abstract samples to even out classes
dataset = dataset[dataset[:, 0].argsort()]
datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]
  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]

dataset=datasetnew

In [ ]:
#Testing Tokenization

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer('basic_english')

########## ADDED

import spacy
from torchtext.data.utils import get_tokenizer

# Create a custom tokenizer using spacy
basic_english_tokenizer = get_tokenizer('basic_english')

# Define a custom tokenizer that also splits on dashes
def custom_tokenizer(text):
    # Use the basic_english tokenizer and split on dashes
    tokens = basic_english_tokenizer(text)
    # Flatten the list of tokens and split on dashes
    tokens = [subtoken for token in tokens for subtoken in token.split("-")]
    return tokens

# Example usage
text = "This is a test-sentence with a dash."
tokens = custom_tokenizer(text)
tokenizer=custom_tokenizer

##############

x,y = dataset.shape

traindata = dataset[:,1]
trainlabel = dataset[:,0]

#get the minimum length of tokenized abstract sample in the training data
min = 100000000000
for i in range(x):
  if len(tokenizer(traindata[i])) <min:
    min = len(tokenizer(traindata[i]))

testdata = testdataset[:,1]
testlabel = testdataset[:,0]
x,y = testdataset.shape

#get the minimum length of tokenized abstract sample in the test data
min = 100000000000
for i in range(x):
  if len(tokenizer(testdata[i])) <min:
    min = len(tokenizer(testdata[i]))


In [ ]:
#Tokenizing Training & Testing Data

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer('basic_english')

WORDlen = 40

class CustomTextDataset(Dataset): #gives the tokenized abstracts and corresponding labels
    def __init__(self, txt, labels):
        self.labels = labels
        self.text = txt

    def __len__(self):
        return len(self.labels) #number of samples in the data

    def __getitem__(self, idx):
        label = self.labels[idx]-1
        text = self.text[idx]

        text = tokenizer(text) #tokenize the text
        text = np.array(text) #convert to numpy array

        text = text[:WORDlen] #if length is greater than WORDlen, crop tokenized text to WORDlen limit

        while np.shape(text)[0] < WORDlen:
            text = np.append(text, ["hjgjhgjhgfgdgfdgfdfjhj"]) #if length is less than WORDlen, build up to length

        text = list(text)

        sample = {"Text": text, "Class": label} #return tokenized text and labels
        return sample

traindata_set = CustomTextDataset(traindata, trainlabel)
testdata_set = CustomTextDataset(testdata, testlabel)


{'Text': ['a', 'hypothesis', 'nonsteroidal', 'anti-inflammatory', 'drugs', 'reduce', 'the', 'incidence', 'of', 'large-bowel', 'cancer', '.', 'nonsteroidal', 'anti-inflammatory', 'drugs', '(', 'nsaids', ')', 'inhibit', 'prostaglandin'], 'Class': 0}


In [ ]:
#Data Load --> getting shuffled batches for training

trainloader = DataLoader(traindata_set, batch_size = 64, shuffle = True)
trainloader2 = DataLoader(traindata_set, batch_size = 4000, shuffle = True)
testloader2 = DataLoader(testdata_set, batch_size = 2000, shuffle = True)

#parameters
num_epochs = 40

#for epochs in range(num_epochs):
"""
for i in trainloader:
  print(i)

for idx,D in enumerate(trainloader):
    print(D)
    break
"""

for (idx, batch) in enumerate(trainloader):
    # Print the 'text' data of the batch
    print(np.array(batch['Text']).T[0])
    print(idx, 'Text data: ', (batch['Text'])[0][1])
    # Print the 'class' data of batch
    print(idx, 'Class data: ', len(batch['Class']), '\n')

    break

86
['update' 'prostate' 'cancer' '.' 'prostate' 'cancer' 'represents' 'a'
 'major' 'health-care' 'issue' 'for' 'older' 'men' '.' 'recent'
 'technical' 'advances' 'in' 'imaging']
0 Text data:  early
0 Class data:  64 



In [ ]:
#Building the Vocabulary
#Our current implementation of the vocabulary is all words that are seen in training and testing data.

def build_vocab(first, second, WORDlen): #WORDlen = 20 (from above)
  L = []

  if first: #words from train
    for i in traindata:
      L.append(tokenizer(i)[:WORDlen]) #get cropped tokenized abstract

  if second: #words from test
    for i in testdata:
      L.append(tokenizer(i)[:WORDlen]) #get cropped tokenized abstract

  L=np.array(L)
  L=np.concatenate(L)

  return list(L)

test_vocab_list=dict()
train_vocab_list=dict()
train_tokens = list(set(build_vocab(True, True, WORDlen)))
#print(len(train_tokens))

"""
for wordlength in [50,100]:
  z=(len(list(set(build_vocab(True,True,wordlength))))) #a+b+c
  x=(len(list(set(build_vocab(True,False,wordlength)))))  #a+c
  y=(len(list(set(build_vocab(False,True,wordlength))))) #b+c
  c=x+y-z
  print(y,y-c)
"""
"""
for wordlength in [50]:
  train_tokens_1 = list(set(build_vocab(True,False,wordlength)))
  test_tokens_1 = list(set(build_vocab(False,True,wordlength)))
  test_vocab_list = dict(zip(test_tokens_1, range(len(test_tokens_1))))
  train_vocab_list = dict(zip(train_tokens_1, range(len(train_tokens_1))))

count_test=0
count_train=0
total=0

for word in test_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_test+=1
  total+=1
  x = modelemb.wv.get_vector(word)

  #if word not in train_vocab_list:
  #  print(word)
for word in train_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_train+=1
  total+=1
print(count_test/total)
print(count_train/total)

"""
#vocab list: dictionary made from vocabulary
vocab_list = dict(zip(train_tokens, range(len(train_tokens))))

'''
vocab_list_one_hot = dict()
for i in vocab_list:
  length = len(vocab_list)
  one_hot = np.zeros(length)
  vocab_list[i]
  one_hot[vocab_list[i]] = 1
  vocab_list_one_hot[i] = one_hot'''


In [ ]:
#LSTM Implementation

import torch.nn as nn
class MyNeuralNet(nn.Module):

    def __init__(self):
        super(MyNeuralNet,self).__init__()

        #self.embedding = nn.Embedding(int(len(vocab_list)),250)
        #self.rnn = nn.RNN(600,128,1,batch_first=True)

        self.rnn = torch.nn.LSTM(300, 100, 1, batch_first=True, dropout=.2) #medical embedding size = 300
        self.flatten = nn.Flatten(1,2)
        inp=100

        self.dropout = nn.Dropout(.3)
        self.hid_layer = nn.Linear(inp*WORDlen, 100) #WORDlen = 20 (from above)
        self.dropout2 = nn.Dropout(.3)
        self.out_layer = nn.Linear(inp*WORDlen, 5)
        self.sm = nn.Softmax(dim = 1)
        #self.multihead_attn = nn.MultiheadAttention(embed_dim=, num_heads=2)

    def forward(self,x):
        #x =self.embedding(x)
        x, x2=self.rnn(x)
        #print(x2.shape,x.shape)
        #x = torch.swapaxes(x,0,1)

        #x=x[WORDlen-1,:,:]
        #print(x.shape)

        x=self.flatten(x)
        x=self.dropout(x)
        #x=self.hid_layer(x)
        #x=self.dropout2(x)
        #attn_output, attn_output_weights = multihead_attn(query, key, value)
        x=self.out_layer(x)
        yhat=self.sm(x)

        #N 27 128  switch axes 27 N 128 --> 0 N 128

        return x, yhat


In [ ]:
#Preparing Training Data w Embeddings
#After we have tokenized and shuffled batches of our training data, we need to prepare data for training in the LSTM,
#by converting each word in the tokenized text to a 300d embedding.
#We use the Clinical Embeddings here: https://github.com/gweissman/clinical_embeddings/blob/master/README.md.

def prepareXY(batchsize,batch):
        #x = torch.from_numpy(np.zeros((batchsize,WORDlen)))
        #x = torch.from_numpy(np.zeros((batchsize,WORDlen,300)))

        #randomized tensor to fill
        x = (-2)*torch.rand((batchsize, WORDlen, 300))+1 #embedding size = 300, WORDlen = 20, batchsize = 64

        x2 = np.array(batch['Text']).T #training data

        for i2 in range(batchsize):
          for j2 in range(WORDlen): #27 --> 206

            #x[i2,j2] = vocab_list[x2[i2][j2]] #"the"->50
            word=x2[i2][j2]
            if modelemb.wv.has_index_for(word): #if there exists a clinical embedding, convert word to embedding
              x[i2,j2] = torch.from_numpy(modelemb.wv.get_vector(word))

            else: x[i2,j2]=(-2)*torch.rand(300)+1 #else, introduce a random 300d embedding for the word

        x = torch.tensor(x).to(torch.float32)

        y = np.array(batch['Class'])
        y = torch.from_numpy(y.astype(int))

        #one hot vectors for labels
        yactual = []
        for j in range(batchsize):
          one_hot = np.zeros(5)
          one_hot[y[j]] = 1
          yactual.append(one_hot)

        yactual = torch.Tensor(yactual)
        return x, yactual

#Get model accuracy
def accuracyfun(batchsize,q,yactual):
        accuracy = 0
        for i in range(batchsize):
          if torch.argmax(q[i]) == torch.argmax(yactual[i]):
            accuracy += 1

        accuracy/=batchsize
        return accuracy


In [ ]:
#LSTM Training Loop

loss_fn = nn.CrossEntropyLoss()
model = MyNeuralNet()
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
trainlosses, testlosses, testaccuracies, trainaccuracies = [],[],[],[]

#model.to(device)
for epoch in range(40): #

    for (idx, batch) in enumerate(trainloader):
    # Print the 'text' data of the batch

    # Print the 'class' data of batch
        if idx>80: break
        x,yactual = prepareXY(64,batch)
        p,q=model(x)

        loss =loss_fn(p,yactual)
        accuracy = accuracyfun(64,q,yactual)

        #print(model.state_dict())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    for idx, batch in enumerate(trainloader2):

        x,yactual = prepareXY(4000, batch)
        p,q=model(x)
        loss =loss_fn(p,yactual)

        accuracy = accuracyfun(4000, q, yactual)
        counts=[0,0,0,0,0]

        for i in range(4000):
          counts[torch.argmax(q[i])-1]+=1

        print(counts)
        break

    for idx, batch in enumerate(testloader2):
        x,yactual = prepareXY(2000,batch)
        p,q=model(x)
        testloss =loss_fn(p,yactual)

        testaccuracy=accuracyfun(2000,q,yactual)

        break

    trainlosses.append(loss.item())
    testlosses.append(testloss.item())
    testaccuracies.append(testaccuracy)
    trainaccuracies.append(accuracy)
    print(loss.item(),accuracy,testloss.item(),testaccuracy)

print("train:", trainlosses, testlosses)
print("test:", testaccuracies, trainaccuracies)
